In [ ]:
# pip install polars-lts-cpu
# ДОБАВЬТЕ ЭТО В САМУЮ ПЕРВУЮ ЯЧЕЙКУ NOTEBOOK
import os
os.environ['POLARS_SKIP_CPU_CHECK'] = '1'

import polars as pl
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# import seaborn as sns
from pathlib import Path
import gc
import psutil
import os
import json
from datetime import datetime

## Этап 1: Детальное исследование данных (EDA) и проектирование фич

In [10]:
# Чтение файла ml_ozon_recsys_test_for_participants
df = pd.read_parquet('/home/dima/ozon/data/ml_ozon_recsys_test_for_participants/test_for_participants/created_date=2025-07-03/part-00000-3020be67-2ce5-4232-b8de-251f063b386a.c000.snappy.parquet', engine='fastparquet')
df.head()

,item_id,user_id,created_timestamp
0,119241,1156930,2025-07-03 08:30:48.743
1,350526,3772191,2025-07-03 12:51:19.050
2,427645,4953991,2025-07-03 04:09:18.837
3,617762,3048491,2025-07-03 16:15:38.360
4,936816,1079151,2025-07-03 15:21:25.660


In [11]:
# Чтение файла ml_ozon_recsys_train_final_apparel_items_data
df = pd.read_parquet('/home/dima/ozon/data/ml_ozon_recsys_train_final_apparel_items_data/ml_ozon_recsys_train_final_apparel_items_data/part-00000-5322ad28-d73d-4c69-86d2-4efc7ad4f422-c000.snappy.parquet', engine='fastparquet')
df.head()

,item_id,itemname,attributes,fclip_embed,catalogid,variant_id,model_id
0,400301,Очки солнцезащитные,None,"[0.32023853, 0.11766258, -0.784552, 0.4168199,...",17080,243954236,33306429
1,678602,Футболка GOLDUSTIM Футболка женская,None,"[0.4360095, 0.6339668, 0.029741853, 0.09847632...",7508,8039633,27057398
2,746401,Худи Patagonia,None,"[0.47452784, -0.4727797, -0.51971936, -0.08221...",7555,120078071,11645051
3,872829,"Комплект носков adidas 3S C Spw Mid 3P, 3 пары",None,"[-0.06987186, -2.2802832, -0.5829071, -0.33674...",36563,247120069,38088455
4,893255,Жилет Buy & Style жилеты,None,"[0.12488763, 0.5466293, 0.29031003, 0.21499962...",7535,327822613,19339929


In [12]:
# Чтение файла ml_ozon_recsys_train_final_apparel_orders_data
df = pd.read_parquet('/home/dima/ozon/data/ml_ozon_recsys_train_final_apparel_orders_data/ml_ozon_recsys_train_final_apparel_orders_data/part-00000-7b718cf5-560c-4ace-bbb7-e36ef433637b-c000.snappy.parquet', engine='fastparquet')
df.head()

,item_id,user_id,created_timestamp,last_status,last_status_timestamp,created_date
0,15535,3423970,2025-01-11 19:20:02.550,delivered_orders,2025-01-16 10:53:03,2025-01-11
1,15535,1207601,2025-03-25 13:11:48.240,canceled_orders,2025-03-29 08:32:43,2025-03-25
2,119241,2340021,2025-05-03 20:46:18.146,canceled_orders,2025-05-12 17:10:27,2025-05-03
3,119241,2356390,2025-05-01 17:53:00.716,delivered_orders,2025-05-14 14:36:20,2025-05-01
4,119241,1490080,2025-04-22 22:59:11.160,canceled_orders,2025-04-26 06:27:28,2025-04-22


In [13]:
# Чтение файла ml_ozon_recsys_train_final_apparel_tracker_data
df = pd.read_parquet('/home/dima/ozon/data/ml_ozon_recsys_train_final_apparel_tracker_data/ml_ozon_recsys_train_final_apparel_tracker_data/part-00000-582d88ee-8dd6-41f6-9f90-375910468cb7-c000.snappy.parquet', engine='fastparquet')
df.head()

,item_id,user_id,timestamp,action_type,action_widget,date
0,678602,4211601,2025-05-05 18:11:15,page_view,cart.cartSplit,2025-05-05
1,678602,4211601,2025-05-05 18:12:57,view_description,pdp.characteristics,2025-05-05
2,678602,4211601,2025-05-05 18:12:52,view_description,pdp.richContent,2025-05-05
3,678602,4211601,2025-05-05 10:39:08,page_view,cart.cartSplit,2025-05-05
4,678602,4211601,2025-05-05 18:12:57,view_description,pdp.characteristics,2025-05-05


In [14]:
# Чтение файла ml_ozon_recsys_train_final_categories_tree
df = pd.read_parquet('/home/dima/ozon/data/ml_ozon_recsys_train_final_categories_tree/ml_ozon_recsys_train_final_categories_tree/part-00000-163f9108-23d6-4fd6-98c4-7bcb73ee27e2-c000.snappy.parquet', engine='fastparquet')
df.head()

,catalogid,catalogpath,ids
0,15500,None,"[15500, -1]"
1,15501,None,"[15501, 15500, -1]"
2,15502,None,"[15502, 15501, 15500, -1]"
3,15511,None,"[15511, 15501, 15500, -1]"
4,15892,None,"[15892, 15511, 15501, 15500, -1]"


In [15]:
# Чтение файла ml_ozon_recsys_test.snappy
df = pd.read_parquet('/home/dima/ozon/data/ml_ozon_recsys_test_for_participants/test_for_participants/ml_ozon_recsys_test.snappy.parquet', engine='fastparquet')
df.head()

,user_id
0,1857340
1,4094281
2,4815130
3,38621
4,3530170


In [ ]:
plt.style.use('ggplot')
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

class OzonDataAnalyzer:
    def __init__(self, data_path: str):
        self.data_path = Path(data_path)
        self.dfs = {}
        self.results = {}
        
    def filter_valid_parquet_files(self, file_paths):
        """Фильтруем только настоящие parquet файлы (исключаем служебные)"""
        valid_files = []
        for file_path in file_paths:
            # Исключаем файлы macOS и слишком маленькие файлы
            if not file_path.name.startswith('._') and file_path.stat().st_size > 1024:  # > 1KB
                valid_files.append(file_path)
        
        print(f"   Найдено {len(valid_files)} валидных файлов (исключены служебные)")
        return valid_files
        
    def find_all_parquet_files(self, folder_name):
        """Рекурсивный поиск всех parquet файлов в папке"""
        folder_path = self.data_path / folder_name
        if not folder_path.exists():
            print(f"✗ Папка {folder_name} не существует")
            return []
            
        # Рекурсивно ищем все .parquet файлов
        parquet_files = list(folder_path.rglob("*.parquet"))
        print(f"{folder_name}: найдено {len(parquet_files)} .parquet файлов")
        
        # Фильтруем только валидные файлы
        valid_files = self.filter_valid_parquet_files(parquet_files)
        
        # Показываем структуру
        for i, file_path in enumerate(valid_files[:3]):
            size_mb = file_path.stat().st_size / 1024 / 1024
            print(f"   {i+1}. {file_path.name} ({size_mb:.1f} MB)")
        if len(valid_files) > 3:
            print(f"   ... и еще {len(valid_files) - 3} файлов")
            
        return valid_files
        
    def safe_read_parquet(self, file_path, n_rows=None, columns=None):
        """Безопасное чтение parquet файла"""
        try:
            size_mb = file_path.stat().st_size / 1024 / 1024
            print(f"   Чтение: {file_path.name} ({size_mb:.1f} MB)")
            
            # Читаем сначала схему для отладки
            schema = pl.read_parquet(file_path, n_rows=0).schema
            print(f"   Схема: {list(schema.keys())}")
            
            # Используем use_pyarrow=False для совместимости с n_rows
            df = pl.read_parquet(
                file_path,
                n_rows=n_rows,
                columns=columns,
                use_pyarrow=False,
                low_memory=True
            )
            print(f"  Успешно: {len(df)} строк, {len(df.columns)} столбцов")
            return df
            
        except Exception as e:
            print(f"  Ошибка чтения {file_path.name}: {e}")
            return None
            
    def read_multiple_parquet_files(self, file_paths, n_rows=None, columns=None, max_files=3):
        """Чтение нескольких parquet файлов с проверкой совместимости схем"""
        if not file_paths:
            return None
            
        print(f"Чтение первых {min(len(file_paths), max_files)} файлов из {len(file_paths)}...")
        
        all_dfs = []
        first_schema = None
        
        for i, file_path in enumerate(file_paths[:max_files]):
            df = self.safe_read_parquet(file_path, n_rows, columns)
            if df is not None:
                # Проверяем совместимость схем
                if first_schema is None:
                    first_schema = df.schema
                    all_dfs.append(df)
                    print(f"  Используем схему: {list(first_schema.keys())}")
                else:
                    # Проверяем, совпадают ли схемы
                    current_schema = df.schema
                    if first_schema == current_schema:
                        all_dfs.append(df)
                    else:
                        print(f"  Пропускаем файл {file_path.name} - несовместимая схема")
                        print(f"  Ожидалось: {list(first_schema.keys())}")
                        print(f"  Получено: {list(current_schema.keys())}")
                        
        if all_dfs:
            try:
                return pl.concat(all_dfs)
            except Exception as e:
                print(f"Ошибка при объединении DataFrame: {e}")
                # Возвращаем первый DataFrame если объединение не удалось
                return all_dfs[0] if all_dfs else None
        return None
        
    def load_sample_data(self):
        """Загрузка выборки данных с рекурсивным поиском"""
        print("Загрузка данных с рекурсивным поиском...")
        
        datasets = [
            ('test', "ml_ozon_recsys_test_for_participants", 5000, None, 2),
            ('categories', "ml_ozon_recsys_train_final_categories_tree", None, None, 1),
            ('items', "ml_ozon_recsys_train_final_apparel_items_data", 3000, 
             ['item_id', 'itemname', 'catalogid', 'variant_id', 'model_id'], 2),
            ('orders', "ml_ozon_recsys_train_final_apparel_orders_data", 2000,
             ['item_id', 'user_id', 'last_status', 'created_timestamp'], 2),
            ('tracker', "ml_ozon_recsys_train_final_apparel_tracker_data", 1000,
             ['item_id', 'user_id', 'timestamp', 'action_type'], 2)
        ]
        
        for df_name, folder, n_rows, columns, max_files in datasets:
            print(f"\nПоиск файлов в {folder}...")
            
            # Находим все parquet файлы рекурсивно
            parquet_files = self.find_all_parquet_files(folder)
            
            if parquet_files:
                # Читаем несколько файлов
                df = self.read_multiple_parquet_files(parquet_files, n_rows, columns, max_files)
                if df is not None:
                    self.dfs[df_name] = df
                    print(f"✓ {folder}: загружено {len(df)} строк из {max_files} файлов")
                else:
                    print(f"✗ {folder}: не удалось загрузить данные")
            else:
                print(f"✗ {folder}: parquet файлы не найдены")
            
            gc.collect()
    
    def analyze_test_data(self):
        """Анализ тестовых данных"""
        if 'test' not in self.dfs or self.dfs['test'] is None:
            print("✗ Тестовые данные не загружены")
            return {}
            
        df = self.dfs['test']
        print("Анализ тестовых данных.")
        
        result = {
            'rows': len(df),
            'columns': df.columns,
            'available_columns': list(df.columns)
        }
        
        # Проверяем наличие столбцов перед анализом
        if 'user_id' in df.columns:
            result['unique_users'] = df['user_id'].n_unique()
        
        if 'item_id' in df.columns:
            result['unique_items'] = df['item_id'].n_unique()
        else:
            print("  Столбец 'item_id' не найден в тестовых данных")
            result['unique_items'] = 'not_available'
        
        if 'created_timestamp' in df.columns:
            result['time_range'] = {
                'min': str(df['created_timestamp'].min()),
                'max': str(df['created_timestamp'].max())
            }
        else:
            print("  Столбец 'created_timestamp' не найден в тестовых данных")
            result['time_range'] = 'not_available'
        
        return result
    
    def show_test_structure(self):
        """Показать структуру тестовых данных"""
        if 'test' in self.dfs and self.dfs['test'] is not None:
            df = self.dfs['test']
            print("Структура тестовых данных:")
            print(f"Колонки: {df.columns}")
            print(f"Первые 5 строк:")
            print(df.head())
            print(f"Схема: {df.schema}")
        else:
            print("Тестовые данные не загружены")
    
    def analyze_items_data(self):
        """Анализ данных товаров"""
        if 'items' not in self.dfs or self.dfs['items'] is None:
            print("✗ Данные товаров не загружены")
            return {}
            
        df = self.dfs['items']
        print("Анализ данных товаров.")
        return {
            'rows': len(df),
            'unique_categories': df['catalogid'].n_unique(),
            'null_names': df['itemname'].null_count(),
            'unique_names': df['itemname'].n_unique(),
            'columns': df.columns
        }
    
    def analyze_orders_data(self):
        """Анализ данных заказов"""
        if 'orders' not in self.dfs or self.dfs['orders'] is None:
            print("✗ Данные заказов не загружены")
            return {}
            
        df = self.dfs['orders']
        print("Анализ данных заказов.")
        
        result = {
            'rows': len(df),
            'columns': df.columns
        }
        
        # Проверяем наличие обязательных столбцов
        if 'last_status' in df.columns:
            status_counts = {}
            for status in df['last_status'].unique():
                count = df.filter(pl.col('last_status') == status).height
                status_counts[status] = count
            
            result['status_distribution'] = status_counts
            result['delivered_orders'] = status_counts.get('delivered_orders', 0)
            result['canceled_orders'] = status_counts.get('canceled_orders', 0)
        else:
            print("  Столбец 'last_status' не найден")
            result['status_distribution'] = 'not_available'
        
        if 'user_id' in df.columns:
            result['unique_users'] = df['user_id'].n_unique()
        else:
            result['unique_users'] = 'not_available'
        
        if 'item_id' in df.columns:
            result['unique_items'] = df['item_id'].n_unique()
        else:
            result['unique_items'] = 'not_available'
        
        if 'created_timestamp' in df.columns:
            result['time_range'] = {
                'min': str(df['created_timestamp'].min()),
                'max': str(df['created_timestamp'].max())
            }
        else:
            result['time_range'] = 'not_available'
        
        return result

    def analyze_tracker_data(self):
        """Анализ данных трекера"""
        if 'tracker' not in self.dfs or self.dfs['tracker'] is None:
            print("✗ Данные трекера не загружены")
            return {}
            
        df = self.dfs['tracker']
        print("Анализ данных трекера.")
        
        result = {
            'rows': len(df),
            'columns': df.columns
        }
        
        if 'action_type' in df.columns:
            action_counts = {}
            for action in df['action_type'].unique():
                count = df.filter(pl.col('action_type') == action).height
                action_counts[action] = count
            
            result['action_distribution'] = action_counts
        else:
            print("  Столбец 'action_type' не найден")
            result['action_distribution'] = 'not_available'
        
        if 'user_id' in df.columns:
            result['unique_users'] = df['user_id'].n_unique()
        else:
            result['unique_users'] = 'not_available'
        
        if 'item_id' in df.columns:
            result['unique_items'] = df['item_id'].n_unique()
        else:
            result['unique_items'] = 'not_available'
        
        if 'timestamp' in df.columns:
            result['time_range'] = {
                'min': str(df['timestamp'].min()),
                'max': str(df['timestamp'].max())
            }
        else:
            result['time_range'] = 'not_available'
        
        return result
    
    def analyze_categories_data(self):
        """Анализ дерева категорий"""
        if 'categories' not in self.dfs or self.dfs['categories'] is None:
            print("✗ Данные категорий не загружены")
            return {}
            
        df = self.dfs['categories']
        print("Анализ дерева категорий.")
        
        # Безопасное вычисление максимальной глубины
        max_depth = 0
        for ids in df['ids']:
            if ids is not None:
                max_depth = max(max_depth, len(ids))
        
        return {
            'rows': len(df),
            'total_categories': df['catalogid'].n_unique(),
            'max_depth': max_depth,
            'columns': df.columns
        }
    
    def create_basic_visualizations(self):
        """Создание отдельных визуализаций для каждого доступного типа данных"""
        try:
            created_plots = 0
            
            # 1. Распределение статусов заказов
            if ('orders' in self.dfs and self.dfs['orders'] is not None and 
                'last_status' in self.dfs['orders'].columns):
                
                df = self.dfs['orders']
                status_counts = {}
                for status in df['last_status'].unique():
                    count = df.filter(pl.col('last_status') == status).height
                    status_counts[status] = count
                
                if status_counts:
                    plt.figure(figsize=(10, 7))
                    labels = list(status_counts.keys())
                    sizes = list(status_counts.values())
                    plt.pie(sizes, labels=labels, autopct='%1.1f%%')
                    plt.title('Распределение статусов заказов')
                    plt.tight_layout()
                    plt.savefig('orders_status_distribution.png', dpi=150, bbox_inches='tight')
                    plt.close()
                    created_plots += 1
                    print("Создана визуализация статусов заказов")
            
            # 2. Распределение действий в трекере
            if ('tracker' in self.dfs and self.dfs['tracker'] is not None and 
                'action_type' in self.dfs['tracker'].columns):
                
                df = self.dfs['tracker']
                action_counts = {}
                for action in df['action_type'].unique():
                    count = df.filter(pl.col('action_type') == action).height
                    action_counts[action] = count
                
                if action_counts:
                    plt.figure(figsize=(10, 7))
                    # Берем топ-5 действий
                    sorted_actions = sorted(action_counts.items(), key=lambda x: x[1], reverse=True)[:5]
                    actions, counts = zip(*sorted_actions)
                    
                    plt.bar(range(len(actions)), counts)
                    plt.xticks(range(len(actions)), actions, rotation=45)
                    plt.title('Топ-5 типов действий в трекере')
                    plt.tight_layout()
                    plt.savefig('tracker_actions_distribution.png', dpi=150, bbox_inches='tight')
                    plt.close()
                    created_plots += 1
                    print("Создана визуализация действий трекера")
            
            if created_plots == 0:
                print("Недостаточно данных для визуализаций")
                return False
            
            print(f"Создано {created_plots} визуализаций")
            return True
            
        except Exception as e:
            print(f"Ошибка создания визуализаций: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def design_features(self):
        """Проектирование фич для рекомендательной системы"""
        feature_plan = {
            'user_features': [
                'user_total_events', 'user_unique_items', 'user_activity_recency',
                'user_buying_frequency', 'user_cart_add_rate', 'user_purchase_rate'
            ],
            'item_features': [
                'item_popularity', 'item_recent_popularity', 'item_conversion_rate',
                'item_category', 'item_price_tier', 'item_age'
            ],
            'user_item_features': [
                'user_item_view_count', 'user_item_time_since_last_view',
                'user_item_in_cart', 'user_item_in_wishlist'
            ],
            'context_features': [
                'time_of_day', 'day_of_week', 'season', 'is_weekend'
            ]
        }
        
        return feature_plan
    
    def generate_feature_engineering_plan(self):
        """Генерация плана feature engineering"""
        return {
            'steps': [
                '1. Подготовка данных и очистка',
                '2. Генерация user features из tracker данных',
                '3. Генерация item features из items и tracker данных', 
                '4. Создание user-item interaction features',
                '5. Добавление временных и контекстных features',
                '6. Negative sampling и создание целевой переменной'
            ],
            'recommendations': [
                'Использовать Polars для эффективной обработки',
                'Применить временную валидацию',
                'Использовать инкрементальную обработку для больших данных'
            ]
        }
    
    def get_summary_statistics(self):
        """Получение сводной статистики"""
        # Фильтруем только не-None датасеты
        valid_datasets = {name: df for name, df in self.dfs.items() if df is not None}
        
        return {
            'datasets': {name: len(df) for name, df in valid_datasets.items()},
            'memory_usage_mb': psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024,
            'loaded_datasets_count': len(valid_datasets),
            'total_rows': sum(len(df) for df in valid_datasets.values())
        }
    
    def run_complete_analysis(self):
        """Полный анализ данных"""
        print("Запуск полного анализа данных с рекурсивным поиском")
        print("=" * 60)
        
        try:
            # 1. Загрузка данных
            self.load_sample_data()
            
            # 2. Анализ каждого датасета
            self.results['test_analysis'] = self.analyze_test_data()
            self.results['items_analysis'] = self.analyze_items_data()
            self.results['orders_analysis'] = self.analyze_orders_data()
            self.results['tracker_analysis'] = self.analyze_tracker_data()
            self.results['categories_analysis'] = self.analyze_categories_data()
            self.results['test_analysis'] = self.analyze_test_data()
            
            # 3. Визуализации
            self.results['visualizations_created'] = self.create_basic_visualizations()
            
            # 4. Проектирование фич
            self.results['feature_plan'] = self.design_features()
            self.results['engineering_plan'] = self.generate_feature_engineering_plan()
            
            # 5. Сводная статистика
            summary = self.get_summary_statistics()
            self.results['summary'] = {
                'total_datasets_loaded': summary['loaded_datasets_count'],
                'total_rows_analyzed': summary['total_rows'],
                'analysis_timestamp': datetime.now().isoformat(),
                'memory_usage_mb': summary['memory_usage_mb']
            }
            
            print("Анализ успешно завершен!")
            return self.results
            
        except Exception as e:
            print(f"Ошибка во время анализа: {e}")
            import traceback
            traceback.print_exc()
            return {'error': str(e)}


if __name__ == "__main__":
    # Добавляем фикс для AVX проблемы
    import os
    os.environ['POLARS_SKIP_CPU_CHECK'] = '1'
    
    analyzer = OzonDataAnalyzer("/home/dima/ozon/data")
    results = analyzer.run_complete_analysis()
    
    # Сохранение результатов
    with open('eda_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print("Результаты сохранены в eda_results.json")
    
    # Вывод сводной информации
    if 'summary' in results:
        summary = results['summary']
        print(f"\nСводная статистика:")
        print(f"Проанализировано датасетов: {summary.get('total_datasets_loaded', 0)}") # type: ignore
        print(f"Общее количество строк: {summary.get('total_rows_analyzed', 0)}") # type: ignore
        print(f"Пиковое использование памяти: {summary.get('memory_usage_mb', 0):.1f} MB") # type: ignore
    else:
        print("Не удалось получить сводную статистику")

Запуск полного анализа данных с рекурсивным поиском
Загрузка данных с рекурсивным поиском...

Поиск файлов в ml_ozon_recsys_test_for_participants...
ml_ozon_recsys_test_for_participants: найдено 141 .parquet файлов
   Найдено 71 валидных файлов (исключены служебные)
   1. ml_ozon_recsys_test.snappy.parquet (1.8 MB)
   2. part-00003-3020be67-2ce5-4232-b8de-251f063b386a.c000.snappy.parquet (0.2 MB)
   3. part-00000-3020be67-2ce5-4232-b8de-251f063b386a.c000.snappy.parquet (0.2 MB)
   ... и еще 68 файлов
Чтение первых 2 файлов из 71...
   Чтение: ml_ozon_recsys_test.snappy.parquet (1.8 MB)
   Схема: ['user_id']
  Успешно: 5000 строк, 1 столбцов
  Используем схему: ['user_id']
   Чтение: part-00003-3020be67-2ce5-4232-b8de-251f063b386a.c000.snappy.parquet (0.2 MB)
   Схема: ['item_id', 'user_id', 'created_timestamp']
  Успешно: 5000 строк, 3 столбцов
  Пропускаем файл part-00003-3020be67-2ce5-4232-b8de-251f063b386a.c000.snappy.parquet - несовместимая схема
  Ожидалось: ['user_id']
  Получено